In [37]:
from mstrio.connection import Connection
from mstrio.admin.server import Cluster, ServerSettings
from mstrio.admin.application import Environment, Application, compare_application_settings
from mstrio.admin.usergroup import UserGroup, list_usergroups
from mstrio.admin.user import User, list_users
from mstrio.admin.schedule import ScheduleManager
from mstrio.admin.subscription.subscription import EmailSubscription, Subscription
from mstrio.admin.subscription.subscription_manager import SubscriptionManager
from mstrio.admin.subscription.content import Content
from mstrio.admin.privilege import Privilege
from mstrio.admin.security_role import SecurityRole, list_security_roles
from mstrio.admin.user_connections import UserConnections

from mstrio.library import Library
from mstrio.admin.dossier import list_dossiers
import pandas as pd

# PROD ENVIRONMENT
base_url_prod = "Insert Env URL"
username_prod = "jsmith"
password_prod = "test123"

mstr_tutorial_id = 'B7CA92F04B9FAE8D941C3E9B7E0CD754'

## Connect to Environment

In [38]:
conn_prod = Connection(base_url_prod, username_prod, password_prod, login_mode=1, project_name="MicroStrategy Tutorial")
env_prod = Environment(connection=conn_prod)
tutorial_prod = Application(conn_prod, name="MicroStrategy Tutorial")


Connection to MicroStrategy Intelligence Server has been established.
Application object named: 'MicroStrategy Tutorial' with ID: 'B7CA92F04B9FAE8D941C3E9B7E0CD754'


### Prepare data to connect to database

In [39]:
import pyodbc # Python library to connect to database

driver = '{ODBC Driver 17 for SQL Server}'
server = '127.0.0.1'
database = 'TmpDB'
uid = 'sa'
pwd = 'reallyStrongPwd123'

# prepare string which is used to connect to database
driver = 'DRIVER=' + driver
server = 'SERVER=' + server
database = 'DATABASE=' + database
uid = 'UID=' + uid
pwd = 'PWD=' + pwd
connection_string = ';'.join([driver, server, database, uid, pwd])

### Get newly hired employees from Database and create new user accounts for them. 

In [40]:
query = "SELECT * FROM NEW_HIRE_EMPLOYEE WHERE EMPLOYEE_HIRE_DATE > '12-01-2020';"

with pyodbc.connect(connection_string) as db_conn:
    new_employees = pd.read_sql(query, db_conn)

new_users = []
for _, row in new_employees.iterrows():
    emp_username = row['EMPLOYEE_USER_NAME']
    emp_fullname = row['EMPLOYEE_LEGAL_NAME']
    emp_title = row['EMPLOYEE_TITLE']
    new_usr = User.create(connection=conn_prod, username=emp_username, full_name=emp_fullname,
                          trust_id=emp_username, require_new_password=False)
    new_users.append(new_usr)

Successfully created user named: 'lchase' with ID: '4AF7DE2811EB670DAC050080EF8522E5'
Successfully created user named: 'dayers' with ID: '4B201AAA11EB670DAC050080EF1543E6'
Successfully created user named: 'nmacfarlane' with ID: '4B3EB1EA11EB670DAC050080EF45A3E7'
Successfully created user named: 'mferry' with ID: '4B61970A11EB670DAC050080EFB583E8'
Successfully created user named: 'cphillips' with ID: '4B81BE4B11EB670DAC050080EF3584E9'
Successfully created user named: 'bdaniels' with ID: '4BA0D08211EB670DAC050080EFB584EA'
Successfully created user named: 'brees' with ID: '4BBE4AF511EB670DAC050080EFA562E5'


### Assign to Administrator group

In [41]:
us = UserGroup(connection=conn_prod, name="Administrator")
for usr in new_users:
    usr.add_to_usergroups(usergroups=us)

UserGroup object named: 'Administrator' with ID: 'E96685CD4E60068559F7DFAC7C2AA851'
Added user 'Linda Chase' to group(s): ['Administrator']
Added user 'Dylan Ayers' to group(s): ['Administrator']
Added user 'Nicky Macfarlane' to group(s): ['Administrator']
Added user 'Mark Ferry' to group(s): ['Administrator']
Added user 'Chandler Phillips' to group(s): ['Administrator']
Added user 'Bradley Daniels' to group(s): ['Administrator']
Added user 'Brandon Rees' to group(s): ['Administrator']


***

***

## List all privileges for active users

In [42]:
    all_usrs = new_users
    active_usrs = [u for u in all_usrs if u.enabled]
    lst_prvlgs = []
    #Display user's id, name, username, and privileges
    for usr in active_usrs:
        p = {
            'id': usr.id,
            'name': usr.name,
            'username': usr.username,
            'privileges': usr.privileges
        }
        lst_prvlgs.append(p)

In [43]:
lst_prvlgs

[{'id': '4AF7DE2811EB670DAC050080EF8522E5',
  'name': 'Linda Chase',
  'username': 'lchase',
  'privileges': [{'privilege': {'id': '1',
     'name': 'Create application objects',
     'description': 'This is the privilege required to create an application-level object.',
     'level': 'project'},
    'sources': [{'direct': False,
      'securityRole': {'name': 'System Administrator',
       'id': 'B3D00D8146E0947C1B8C5BAEF87B0B7B'},
      'project': {'name': 'MicroStrategy Tutorial',
       'id': 'B7CA92F04B9FAE8D941C3E9B7E0CD754'},
      'group': {'name': 'Administrator',
       'id': 'E96685CD4E60068559F7DFAC7C2AA851'}}]},
   {'privilege': {'id': '3',
     'name': 'Create schema objects',
     'description': 'This is the privilege required to create a schema-level object.',
     'level': 'project'},
    'sources': [{'direct': False,
      'securityRole': {'name': 'System Administrator',
       'id': 'B3D00D8146E0947C1B8C5BAEF87B0B7B'},
      'project': {'name': 'MicroStrategy Tutoria

***

***

## List Security Roles Per User

In [44]:
def list_security_roles_per_user(connection: "Connection", usergroup_name: str = None,
                                 usergroup_id: str = None, include_subgroups: bool = False):
    
    usrgrp = UserGroup(connection=connection, name=usergroup_name, id=usergroup_id)
    all_scrt_rls = []
    for member in usrgrp.list_members():
        #If the element in the members list is a subgroup, list the subgroups's security role
        if not member.get('full_name', None):
            if not include_subgroups:
                continue
            member_type = 'subgroup'
            tmp_ug = UserGroup(connection=connection, id=member['id'])
            scrt_rls = tmp_ug.security_roles
        #If the element in the members list is a user, list the users's security role
        else:
            member_type = 'user'
            tmp_u = User(connection=connection, id=member['id'])
            scrt_rls = tmp_u.security_roles
        m = {
            'type': member_type,
            'id': member['id'],
            'name': member['name'],
            'username': member.get('username', None),
            'security_roles': scrt_rls
        }
        all_scrt_rls.append(m)
    return all_scrt_rls


In [45]:
list_security_roles_per_user(connection=conn_prod, usergroup_name="Administrator", usergroup_id="E96685CD4E60068559F7DFAC7C2AA851")

UserGroup object named: 'Administrator' with ID: 'E96685CD4E60068559F7DFAC7C2AA851'
User object named: 'Bradley Daniels' with ID: '4BA0D08211EB670DAC050080EFB584EA'
User object named: 'Brandon Rees' with ID: '4BBE4AF511EB670DAC050080EFA562E5'
User object named: 'Chandler Phillips' with ID: '4B81BE4B11EB670DAC050080EF3584E9'
User object named: 'Dylan Ayers' with ID: '4B201AAA11EB670DAC050080EF1543E6'
User object named: 'Linda Chase' with ID: '4AF7DE2811EB670DAC050080EF8522E5'
User object named: 'Mark Ferry' with ID: '4B61970A11EB670DAC050080EFB583E8'
User object named: 'Nicky Macfarlane' with ID: '4B3EB1EA11EB670DAC050080EF45A3E7'


[{'type': 'user',
  'id': '4BA0D08211EB670DAC050080EFB584EA',
  'name': 'Bradley Daniels',
  'username': 'bdaniels',
  'security_roles': [{'name': 'MicroStrategy Tutorial',
    'id': 'B7CA92F04B9FAE8D941C3E9B7E0CD754',
    'securityRoles': [{'name': 'System Administrator',
      'id': 'B3D00D8146E0947C1B8C5BAEF87B0B7B',
      'inherited': True,
      'ancestors': [{'name': 'Administrator',
        'id': 'E96685CD4E60068559F7DFAC7C2AA851'}]},
     {'name': 'Normal Users',
      'id': '73F7482111D3596C60001B8F67019608',
      'inherited': True,
      'ancestors': [{'name': 'Everyone',
        'id': 'C82C6B1011D2894CC0009D9F29718E4F'}]}]},
   {'name': 'Consolidated Education Project',
    'id': 'CE52831411E696C8BD2F0080EFD5AF44',
    'securityRoles': [{'name': 'Normal Users',
      'id': '73F7482111D3596C60001B8F67019608',
      'inherited': True,
      'ancestors': [{'name': 'Everyone',
        'id': 'C82C6B1011D2894CC0009D9F29718E4F'}]}]},
   {'name': 'Hierarchies Project',
    'id': 'B

## Take snapshot of Project/Env settings 

In [46]:
from datetime import datetime
cluster = Cluster(connection=conn_prod)
env_dev_settings = ServerSettings(conn_prod)
now = datetime.now().strftime("%Y-%m-%d__%H:%M")

#Export Environment Settings
env_dev_settings.to_json(f"env_dev_settings_{now}.json")

#Export each Project Settings
apps = cluster.list_applications()
for app in apps:
    a = Application(connection=conn_prod, name = app.name)
    a.settings.to_json(f"{app.name}_{now}.json")

Settings exported to 'env_dev_settings_2021-02-04__12:26.json'
Application object named: 'Consolidated Education Project' with ID: 'CE52831411E696C8BD2F0080EFD5AF44'
Settings exported to 'Consolidated Education Project_2021-02-04__12:26.json'
Application object named: 'Hierarchies Project' with ID: 'B3FEE61A11E696C8BD0F0080EFC58F44'
Settings exported to 'Hierarchies Project_2021-02-04__12:26.json'
Application object named: 'Human Resources Analysis Module' with ID: '4BAE16A340B995CAD24193AA3AC15D29'
Settings exported to 'Human Resources Analysis Module_2021-02-04__12:26.json'
Application object named: 'MicroStrategy Tutorial' with ID: 'B7CA92F04B9FAE8D941C3E9B7E0CD754'
Settings exported to 'MicroStrategy Tutorial_2021-02-04__12:26.json'
Application object named: 'Platform Analytics' with ID: '070EB6B011EB3EDF2F990080EFD54F10'
Settings exported to 'Platform Analytics_2021-02-04__12:26.json'
Application object named: 'Relationships Project' with ID: '4C09350211E69712BAEE0080EFB56D41'
Set

***

***

## Get Dossier Filters

In [47]:
def get_dossier_filters(connection, dossier_id, project_id):
    r = connection.session.get(url=connection.base_url + "/api/v2/dossiers/{}/definition".format(dossier_id),
                               params={"ProjectID": project_id})
    return r

In [48]:
dss = list_dossiers(conn_prod)

f = get_dossier_filters(conn_prod, dossier_id=dss[1].id, project_id="B7CA92F04B9FAE8D941C3E9B7E0CD754")
filters = f.json()['chapters'][0]['filters']
data = []
for entry in filters:
    data.append([dss[1].name, entry['name'],entry['summary']])
    df = pd.DataFrame(data, columns =["Dossier","Attribute/Metric", "Filters"])

df

,Dossier,Attribute/Metric,Filters
0,Campaign Finance,Candidate Party Affiliation,"Candidate Party Affiliation In List DEM, REP"


***

***

***

***

***

***